In [2]:
import numpy as np
from matplotlib import pyplot as plt


def drawPoint(dt,label):
    plt.axis([-1, 1, -1,1])
    positibe = dt[label==1,:]
    plt.scatter(positibe[:,1],positibe[:,2],c = 'r')
    negative = dt[label==-1,:]
    plt.scatter(negative[:,1],negative[:,2],c = 'g')
    plt.show()
    
def buildTargetFunction():
    w0 = 1
    point_1 , point_2 = np.random.uniform(-1., 1., size=(2,2))
    # A*W = b 
    A = [point_1 , point_2]
    b= [-w0,-w0]
    w1, w2 = np.linalg.solve(A, b)
    return np.array([w0, w1, w2])

def generateData(N):
    data =  np.random.uniform(-1,1,size=(N,2))
    data_set =  np.column_stack((np.ones(N), data))
    return data_set

def prediction(data,targetFun):
    labels = np.sign(data.dot(targetFun))
    return labels

def getData(target,N):
    X = generateData(N) 
    Y = prediction(X,target) 
    cntP = len(np.nonzero(Y == 1)[0])
    cntN = len(np.nonzero(Y == -1)[0])
    if cntP < 1 or cntN < 1:
        return getData(target,N)
    return X,Y

In [12]:
# create own SVM
from cvxopt import matrix, solvers
import numpy as np
from sklearn.svm import SVC

def createP(X,Y):
    m,c = X.shape
    P = np.zeros((m,m))
    for i in range(0,m):
        for j in range(0,m):
            P[i,j]= Y[i]*Y[j]*np.dot((X[i,:]).T,X[j,:])
    return P

def solveQP(Q,Y):
    r,_ = Q.shape

    Q = matrix(Q,(r,r))
    p = matrix(-np.ones(r))
    G = matrix(-np.ones(r),(1,r))
    
    h = matrix(0.0)
    A = matrix(Y,(1,r))
    b = matrix(0.0)
    solvers.options['show_progress'] = False
    sol=solvers.qp(Q, p, G, h,A,b)
    return sol['x']

def trainSVM(X_train,Y_train):
    #drawPoint(X_train,Y_train)
    P = createP(X_train,Y_train)
    al = solveQP(P,Y_train)
    x = np.array(al)
    w = np.sum(al  * Y_train.reshape((x.shape[0],1)) * X_train, axis = 0)
    return w

def predict(w,X_test,Y_test):
    yPredict = np.sign(np.dot(X_test,w.T))
    cnt = np.sum(yPredict!=Y_test)
    return (cnt/len(Y_test))

rep = 500
ErrOwnSvn = 0
ErrSvn = 0
countTestPoint = 50
for i in range(0,rep): 
    target = buildTargetFunction()
    X_train,Y_train = getData(target,3)
    X_test,Y_test = getData(target,countTestPoint)
    w = trainSVM(X_train,Y_train)
    ErrOwnSvn += predict(w,X_test,Y_test)
    
    sv = SVC(kernel='linear',C=np.inf).fit(X_train, Y_train)
    yPredict = sv.predict(X_test)
    cnt = np.sum(yPredict!=Y_test)
    ErrSvn+=cnt/countTestPoint
print(ErrOwnSvn/rep)
print(ErrSvn/rep)



0.31116
0.19948
